In [2]:
import numpy as np
import pandas as pd

# load the dataframe
esg = pd.read_csv('../data/esg_metadata.csv')

Show the columns of interest

In [48]:
# create some arrays of essential fields
expl_a_g = list(map(lambda x: 'expl_'+x, list('abcdefg')))
expl_c_g = list(map(lambda x: 'expl_'+x, list('cdefg')))
explans = ['no_gap']+expl_a_g
core_fields = ['sector','cetsid','input_name']+explans

# look just at the no_gap indicators: these should not have any expl variables set
esg[esg['no_gap']==1][core_fields]

,sector,cetsid,input_name,no_gap,expl_a,expl_b,expl_c,expl_d,expl_e,expl_f,expl_g
2,ENV,AG.LND.TOTL.K2,Land area (sq. km),1,0,0,0,0,0,0,0
5,GOV,CC.EST,Control of Corruption: Estimate,1,0,0,1,0,0,0,0
26,ENV,EN.CLC.CDDY.XD,Cooling Degree Days,1,0,0,0,0,0,0,0
28,ENV,EN.CLC.HEAT.XD,Health index 35,1,0,0,0,0,0,0,0
30,ENV,EN.CLC.PRCP.XD,Maximum 5-day Rainfall (25-yr RL),1,0,0,0,0,0,0,0
31,ENV,EN.CLC.SPEI.XD,Mean drought index (SPEI),1,0,0,0,0,0,0,0
32,ENV,EN.MAM.THRD.NO,"Mammal species, threatened",1,0,0,0,0,0,0,0
33,ENV,EN.POP.DNST,Population density (people per sq. km of land ...,1,0,0,0,0,0,0,0
40,ENV,ER.PTD.TOTL.ZS,Terrestrial and marine protected areas (% of t...,1,0,0,0,0,0,0,0
45,GOV,GE.EST,Government Effectiveness: Estimate,1,0,0,1,0,0,0,0


Okay, so we need to do some cleaning so our logic makes sense

In [49]:
# 'no_gap' fields should not have any expl variables set
esg.loc[esg['no_gap']==1,expl_a_g] = 0
# test
esg[esg['no_gap']==1][core_fields]

,sector,cetsid,input_name,no_gap,expl_a,expl_b,expl_c,expl_d,expl_e,expl_f,expl_g
2,ENV,AG.LND.TOTL.K2,Land area (sq. km),1,0,0,0,0,0,0,0
5,GOV,CC.EST,Control of Corruption: Estimate,1,0,0,0,0,0,0,0
26,ENV,EN.CLC.CDDY.XD,Cooling Degree Days,1,0,0,0,0,0,0,0
28,ENV,EN.CLC.HEAT.XD,Health index 35,1,0,0,0,0,0,0,0
30,ENV,EN.CLC.PRCP.XD,Maximum 5-day Rainfall (25-yr RL),1,0,0,0,0,0,0,0
31,ENV,EN.CLC.SPEI.XD,Mean drought index (SPEI),1,0,0,0,0,0,0,0
32,ENV,EN.MAM.THRD.NO,"Mammal species, threatened",1,0,0,0,0,0,0,0
33,ENV,EN.POP.DNST,Population density (people per sq. km of land ...,1,0,0,0,0,0,0,0
40,ENV,ER.PTD.TOTL.ZS,Terrestrial and marine protected areas (% of t...,1,0,0,0,0,0,0,0
45,GOV,GE.EST,Government Effectiveness: Estimate,1,0,0,0,0,0,0,0


In [50]:
# expl_a and expl_b should not have any other variables set
esg.loc[(esg['expl_a']==1) | (esg['expl_b']==1), expl_c_g] = 0

# test
esg[(esg['expl_a']==1) | (esg['expl_b']==1)][core_fields]

,sector,cetsid,input_name,no_gap,expl_a,expl_b,expl_c,expl_d,expl_e,expl_f,expl_g
18,ENV,EN.ATM.CO2E.KT,CO2 emissions (kt),0,0,1,0,0,0,0,0
19,ENV,EN.ATM.CO2E.PC,CO2 emissions (metric tons per capita),0,0,1,0,0,0,0,0
24,ENV,EN.ATM.PM10.MC.M3,"PM10, country level (micrograms per cubic meter)",0,1,0,0,0,0,0,0
27,ENV,EN.CLC.GHGR.MT.CE,GHG net emissions/removals by LUCF (Mt of CO2 ...,0,0,1,0,0,0,0,0
29,ENV,EN.CLC.MDAT.ZS,"Droughts, floods, extreme temperatures (% of p...",0,0,1,0,0,0,0,0
34,ENV,EN.POP.EL5M.ZS,Population living in areas where elevation is ...,0,0,1,0,0,0,0,0
36,ENV,ER.BDV.TOTL.XQ,GEF benefits index for biodiversity (0 = no bi...,0,1,0,0,0,0,0,0
37,ENV,ER.H2O.FWTL.ZS,"Annual freshwater withdrawals, total (% of int...",0,0,1,0,0,0,0,0
38,ENV,ER.H2O.INTR.K3,"Renewable internal freshwater resources, total...",0,0,1,0,0,0,0,0
39,ENV,ER.H2O.INTR.PC,Renewable internal freshwater resources per ca...,0,0,1,0,0,0,0,0


That looks better. Now let's add a field to clearly differentiate WBG indicators from others

In [51]:
esg['domain'] = esg['source_type']
esg.loc[esg['domain']!='WBG','domain'] = 'EXTERNAL'

In [52]:
# Show explanations, disaggregated by domain
esg.groupby(['domain']).sum()[explans]

,no_gap,expl_a,expl_b,expl_c,expl_d,expl_e,expl_f,expl_g
domain,,,,,,,,
EXTERNAL,8,10,8,58,5,8,19,10
WBG,6,5,3,14,0,0,13,2


In [53]:
# limit to the WBG dataset
esg[esg['wbgv1']==1].groupby(['domain']).sum()[explans]

,no_gap,expl_a,expl_b,expl_c,expl_d,expl_e,expl_f,expl_g
domain,,,,,,,,
EXTERNAL,7,0,4,32,3,6,11,5
WBG,6,0,0,8,0,0,7,1


So to me, the big takeaway here is that the big "gaps" are around methodology and household surveys, but we would be very dependent on external sources to make changes, particularly for the WBG dataset

In [54]:
# Disaggregate by sector as well
esg.groupby(['domain','sector']).sum()[explans]

no_gap  expl_a  expl_b  expl_c  expl_d  expl_e  expl_f  \
domain   sector                                                           
EXTERNAL ENV          8       1       8      17       4       8       0   
         GOV          0       3       0       6       0       0       1   
         SOC          0       6       0      35       1       0      18   
WBG      ENV          0       2       3       5       0       0       0   
         GOV          6       0       0       1       0       0       7   
         SOC          0       3       0       8       0       0       6   

                 expl_g  
domain   sector          
EXTERNAL ENV          3  
         GOV          3  
         SOC          4  
WBG      ENV          2  
         GOV          0  
         SOC          0

In [55]:
# just the WBG dataset
esg[esg['wbgv1']==1].groupby(['domain','sector']).sum()[explans]

no_gap  expl_a  expl_b  expl_c  expl_d  expl_e  expl_f  \
domain   sector                                                           
EXTERNAL ENV          7       0       4      13       2       6       0   
         GOV          0       0       0       3       0       0       1   
         SOC          0       0       0      16       1       0      10   
WBG      ENV          0       0       0       2       0       0       0   
         GOV          6       0       0       0       0       0       2   
         SOC          0       0       0       6       0       0       5   

                 expl_g  
domain   sector          
EXTERNAL ENV          2  
         GOV          2  
         SOC          1  
WBG      ENV          1  
         GOV          0  
         SOC          0